# Домашнее задание "Колаборативная фильтрация"

## Библиотеки и дата-сет

https://surpriselib.com/

Задание:

Пакет SURPRISE:

1. используйте данные MovieLens 1M,
можно использовать любые модели из пакета,
получите RMSE на тестовом сете 0,87 и ниже.

2. комментарий преподавателя:
на датасет 1М может не хватить RAM. Можно сделать на 100K. Качество RMSE предлагаю считать на основе Cross-validation (5 фолдов), а не на отложенном датасете.

In [ ]:
import pandas as pd
import numpy as np


In [ ]:
# Датасет Movielens на 1 млн записей (user, movie, rating, timestamp)

!wget 'https://drive.google.com/uc?id=1N3m1bNO1zEf02pp_qLHzUXd6oF2c8UCZ' -O ml-1m.zip


--2024-07-28 18:10:16--  https://drive.google.com/uc?id=1N3m1bNO1zEf02pp_qLHzUXd6oF2c8UCZ
Resolving drive.google.com (drive.google.com)... 74.125.137.139, 74.125.137.101, 74.125.137.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.137.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1N3m1bNO1zEf02pp_qLHzUXd6oF2c8UCZ [following]
--2024-07-28 18:10:16--  https://drive.usercontent.google.com/download?id=1N3m1bNO1zEf02pp_qLHzUXd6oF2c8UCZ
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/octet-stream]
Saving to: ‘ml-1m.zip’

ml-1m.zip           100%[===================>]   5.64M  28.2MB/s    in 0.2s    

2024-07-28 18:10:20 (28.2 MB/s) - 

In [ ]:
!unzip ml-1m.zip

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
# Считываем файлы users, ratings

users_1m = pd.read_csv('./ml-1m/users.dat', sep='::',
                        engine='python',
                        names=['userid', 'gender', 'age', 'occupation', 'zip']).set_index('userid')
ratings_1m = pd.read_csv('./ml-1m/ratings.dat', engine='python',
                          sep='::', names=['userid', 'movieid', 'rating', 'timestamp'])

ratings_1m.movieid = ratings_1m.movieid.astype('int32')
ratings_1m.userid = ratings_1m.userid.astype('int32')
ratings_1m.rating = ratings_1m.rating.astype('float')

In [ ]:
# Файл со списком фильмов берем из датасета на 100K записей

!wget 'https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx' -O MovieLens.zip

--2024-07-28 18:10:32--  https://drive.google.com/uc?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.google.com (drive.google.com)... 74.125.137.139, 74.125.137.101, 74.125.137.113, ...
Connecting to drive.google.com (drive.google.com)|74.125.137.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx [following]
--2024-07-28 18:10:32--  https://drive.usercontent.google.com/download?id=1m0rwReR09achL0xTM6QPoN4tykz5bOMx
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.101.132, 2607:f8b0:4023:c06::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.101.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 847695 (828K) [application/octet-stream]
Saving to: ‘MovieLens.zip’

MovieLens.zip       100%[===================>] 827.83K  --.-KB/s    in 0.1s    

2024-07-28 18:10:35 (8.04 MB/s)

In [ ]:
!unzip MovieLens.zip

Archive:  MovieLens.zip
  inflating: links.csv               
  inflating: movies.csv              
  inflating: ratings.csv             
  inflating: tags.csv                


In [ ]:
#links = pd.read_csv('links.csv')
movies = pd.read_csv('movies.csv')
#ratings = pd.read_csv('ratings.csv')
#tags = pd.read_csv('tags.csv')

In [ ]:
!pip3 install surprise

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357254 sha256=a4410a097a75d5688defb1a627119f67fec5428c40cc237ae7d8fee2ccd44f9f
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [3]:
!pip install surprise

import random

from surprise import accuracy
from surprise import Dataset
from surprise import Reader
from surprise import SVD
from surprise.model_selection import GridSearchCV

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 2.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.4-cp310-cp310-linux_x86_64.whl size=2357288 sha256=99ab36986a07aca397c095e789531627a5a6da8ce94212b5ed1b0f85c3c6763c
  Stored in directory: /root/.cache/pip/wheels/4b/3f/df/6acbf0a40397d9bf3ff97f582cc22fb9ce66adde75bc71fd54
Successfully built scikit-surprise


In [ ]:
from surprise import KNNWithMeans, KNNBasic

from surprise.model_selection import train_test_split

In [ ]:
# Фиксируем случайность
my_seed = 0
random.seed(my_seed)
np.random.seed(my_seed)

In [ ]:
ratings_data = ratings_1m.copy()

In [ ]:
ratings_data.tail(3)

,userid,movieid,rating,timestamp
1000206,6040,562,5.0,956704746
1000207,6040,1096,4.0,956715648
1000208,6040,1097,4.0,956715569


In [ ]:
# Названия столбцов для датасетов на 100K и 1M отличаются

ratings_data.rename(columns={"userid": "userId", "movieid": "movieId", "rating": "rating"}, inplace=True)

In [ ]:
movies_v1 = movies.copy()
movies_v1.head(3)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance




---



## Модель SVD - сингулярное разложение


In [ ]:
# Инициализируем считыватель с диапазоном значения рейтинга от 0.5 до 5

reader = Reader(rating_scale=(0.5, 5))
data = Dataset.load_from_df(ratings_data[["userId", "movieId", "rating"]], reader)

# Загружаем данные из источника в модель и перемешиваем строки
raw_ratings = data.raw_ratings
random.shuffle(raw_ratings)

# 90% train, 10% test
train_test_split_index = int(0.9 * len(raw_ratings))

raw_ratings_train = raw_ratings[:train_test_split_index]
raw_ratings_test = raw_ratings[train_test_split_index:]

# В подборе параметров модели используем только тренировочную часть данных
data.raw_ratings = raw_ratings_train

# Получаем датасет для теста
testset = data.construct_testset(raw_ratings_test)

### Поиск GridSearch по сетке параметров модели SVD


In [ ]:
%%time

# Задаем сетку значений параметров для GridSearch

param_grid = {
    "n_factors": [25, 40],  # default 100, разложение на главные компоненты
    "n_epochs": [10, 20],  # default 20, эпохи
    "lr_all": [0.005, 0.025, 0.125],  # learning rate for all parameters. Default 0.005
    "reg_all": [0.08, 0.16, 0.32],  # regularization term for all parameters. Default 0.02
    "random_state": [0],
}

grid_search = GridSearchCV(
    SVD,
    param_grid,
    measures=["rmse", "mae"],
    cv=5,  # cross-validation
    refit=True,
    n_jobs=-1,
    joblib_verbose=2)

grid_search.fit(data)

pd.DataFrame.from_dict(grid_search.cv_results)[[
    "mean_test_rmse", "mean_test_mae", "param_n_factors",
    "param_n_epochs", "param_lr_all", "param_reg_all"
]].sort_values("mean_test_rmse")

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 158 tasks      | elapsed: 14.7min
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed: 17.2min finished


CPU times: user 4min 27s, sys: 21.3 s, total: 4min 48s
Wall time: 17min 35s


,mean_test_rmse,mean_test_mae,param_n_factors,param_n_epochs,param_lr_all,param_reg_all
30,0.866946,0.685645,40,20,0.025,0.08
12,0.868061,0.686528,25,20,0.025,0.08
21,0.877075,0.694373,40,10,0.025,0.08
3,0.878671,0.695729,25,10,0.025,0.08
27,0.894026,0.708694,40,20,0.005,0.08
9,0.896492,0.710766,25,20,0.005,0.08
31,0.907211,0.722009,40,20,0.025,0.16
13,0.907423,0.722181,25,20,0.025,0.16
18,0.910404,0.722879,40,10,0.005,0.08
0,0.911102,0.723447,25,10,0.005,0.08


In [ ]:
best_model = grid_search.best_estimator["rmse"]

### Предсказания

#### Предсказание для конкретной пары userId - movieId

In [ ]:
best_model.predict(uid=1, iid=1)

Prediction(uid=1, iid=1, r_ui=None, est=4.38363863999267, details={'was_impossible': False})

#### Предсказания для тестового датасета

In [ ]:
testset_predictions = best_model.test(testset)
accuracy.rmse(testset_predictions)

RMSE: 0.8616


0.8615647511464469

#### Лучшие и худшие предсказания

In [ ]:
trainset = best_model.trainset

def get_Iu(uid):
    """ return the number of items rated by given user
    args:
      uid: the id of the user
    returns:
      the number of items rated by the user
    """
    try:
        return len(trainset.ur[trainset.to_inner_uid(uid)])
    except ValueError: # user was not part of the trainset
        return 0

def get_Ui(iid):
    """ return number of users that have rated given item
    args:
      iid: the raw id of the item
    returns:
      the number of users that have rated the item.
    """
    try:
        return len(trainset.ir[trainset.to_inner_iid(iid)])
    except ValueError:  # item was not part of the trainset
        return 0

df = pd.DataFrame(testset_predictions, columns=["uid", "iid", "rui", "est", "details"])
df["Iu"] = df["uid"].apply(get_Iu)
df["Ui"] = df["iid"].apply(get_Ui)
df["err"] = abs(df["est"] - df["rui"])

best_predictions = df.sort_values(by="err")[:5]
worst_predictions = df.sort_values(by="err")[-5:]

In [ ]:
best_predictions

,uid,iid,rui,est,details,Iu,Ui,err
86743,283,720,5.0,5.0,{'was_impossible': False},25,396,0.0
28809,4584,905,5.0,5.0,{'was_impossible': False},53,338,0.0
28793,1437,924,5.0,5.0,{'was_impossible': False},146,1552,0.0
98050,4925,1278,5.0,5.0,{'was_impossible': False},37,1077,0.0
37646,682,1208,5.0,5.0,{'was_impossible': False},62,1058,0.0


In [ ]:
worst_predictions

,uid,iid,rui,est,details,Iu,Ui,err
95887,4737,1218,1.0,4.712140,{'was_impossible': False},100,219,3.712140
99499,296,919,1.0,4.744031,{'was_impossible': False},90,1564,3.744031
52592,4759,924,1.0,4.783605,{'was_impossible': False},40,1552,3.783605
89104,4354,1209,1.0,4.945648,{'was_impossible': False},572,154,3.945648
45133,5416,3147,1.0,4.953611,{'was_impossible': False},40,1102,3.953611


#### Рекомендации

In [ ]:
# Генерируем список рекомендаций для пользователя на основе лучшей модели SVD

dataset = pd.DataFrame({
    'uid': ratings_data.userId,
    'iid': ratings_data.movieId,
    'rating': ratings_data.rating
})


def generate_recommendation(uid, model, dataset, thresh=4, amount=5):

    # uid - userId,
    # thresh - планка рейтинга, выше которой будем отбирать фильмы
    # amount - количество фильмов для рекомендации

    # Определяем фильмы(titles), которые пользователь еще не смотрел

    all_titles = list(dataset['iid'].values)
    users_seen_titles = dataset[dataset['uid'] == uid]['iid']
    titles = np.array(list(set(all_titles) - set(users_seen_titles)))

    # Перемешиваем фильмы
    np.random.shuffle(titles)

    # Формируем список рекомендаций
    rec_list = []
    for title in titles:
        review_prediction = best_model.predict(uid=uid, iid=title)
        rating = review_prediction.est

        # Оставляем только фильмы с рейтингом выше планки
        if rating >= thresh:
            rec_list.append((title, round(rating, 2)))

            # Ограничиваем количество рекомендаций
            if len(rec_list) >= amount:
                if len(rec_list) >= amount:
                  recomend_df = pd.DataFrame(np.asarray(rec_list))
                  recomend_df.rename(columns={0: 'iid', 1: 'rating'}, inplace=True)

                  return recomend_df

In [ ]:
# Подгружаем название фильма к индексам рекомендованых фильмов

recomend_df = generate_recommendation(1, best_model, dataset)
recomend_df.rename(columns={'iid': 'movieId'}, inplace=True)

recomend_df = pd.merge(recomend_df, movies_v1, on='movieId')
recomend_df.sort_values(by='rating', ascending=False)

,movieId,rating,title,genres
0,1219.0,4.34,Psycho (1960),Crime|Horror
1,1678.0,4.28,"Joy Luck Club, The (1993)",Drama|Romance
3,937.0,4.17,Love in the Afternoon (1957),Comedy|Romance
2,2353.0,4.14,Enemy of the State (1998),Action|Thriller
4,1923.0,4.13,There's Something About Mary (1998),Comedy|Romance


#### Итоги:

1. Данные датасет MovieLens 1M (пользователь - фильм - рейтинг)
2. Чтобы рекомендовать пользователю фильм, который он не смотрел можно воспользоваться рейтингами, которые выставляли на этот фильм другие пользователи.
3. Для прогнозирования построена SVD модель коллаборативной фильтрации на основе разложения матрицы 'фильм - пользователь' со значениями поставленного рейтинга
4. Использован GridSearch: поиск по сетке оптимальных параметров.
5. Лучшая модель при кросс-валидации на 5 фолдов показала точность 0.8669 по метрике RMSE на обучающей выборке и 0.8616 на тестовом датасете
6. Выведены лучшие и худшие прогнозы модели для всего датасета
7. Построена функция рекомендации фильмов для пользователя под капотом которой модель SVD с найденными параметрами  